# Day 11: Monkey in the Middle

In [109]:
test_data = [line.strip() for line in open('Input/Day 11 Test.txt')]
real_data = [line.strip() for line in open('Input/Day 11.txt')]
data = test_data.copy()

In [236]:
import re
from tqdm.notebook import tqdm

In [232]:
class Monkey:
    def __init__(self, id):
        self.id = id
        self.counter = 0
    
    def __repr__(self):
        return f"Monkey({self.id})"
    
    def describe(self):
        return f'id:      {self.id}\n' \
             + f'items:   {self.items}\n' \
             + f'oper:    {self.operation}\n' \
             + f'test:    {self.test}\n' \
             + f'actions: {self.true_action}/{self.false_action}'
    
    def turn(self):
        throw_list = [self.process_item(item) for item in self.items]
        self.items = []
        return throw_list
    
    def process_item(self, item):
        item = self.inspect_item(item)
        if self.test_item(item):
            return (self.true_action[1], item)
        else:
            return (self.false_action[1], item)
        
    def test_item(self, item):
        if self.test[0] == 'divisible':
            return item % self.test[1] == 0
        else:
            raise ValueError('Unknown test "{self.test[0]}" for Monkey({self.id})')
        
    def inspect_item(self, item):
        self.counter += 1
        operation = self.operation[0]
        parameter = item if self.operation[1] == 'old' else int(self.operation[1])
        if operation == '*':
            return (item * parameter) // 3
        elif operation == '+':
            return (item + parameter) // 3
        else:
            raise ValueError(f'Unknown operation "{operation} for Monkey({self.id})')

def show_items(monkeys):
    for idx in sorted(monkeys):
        m = monkeys[idx]
        print(f'{m.id:2}: {m.items}')
        
def monkey_round(monkeys):
    for idx in sorted(monkeys):
        m = monkeys[idx]
        throw_list = m.turn()
        # print(f'{idx}: {throw_list}')
        for recipient, item in throw_list:
            monkeys[recipient].items.append(item)

            
class Monkey2(Monkey):
    def inspect_item(self, item):
        self.counter += 1
        operation = self.operation[0]
        parameter = item if self.operation[1] == 'old' else int(self.operation[1])
        if operation == '*':
            return (item * parameter)
        elif operation == '+':
            return (item + parameter)
        else:
            raise ValueError(f'Unknown operation "{operation} for Monkey({self.id})')
    

In [209]:
# parse input data
def parse(data, MClass=Monkey):
    monkeys = {}
    for line in data:
        if match := re.search(r'monkey (\d+):', line, flags=re.I):
            monkey = MClass(int(match.group(1)))
            monkeys[monkey.id] = monkey
        elif match := re.search(r'starting items: ((\d+[,\s]*)+)', line, flags=re.I):
            monkey.items = [int(item) for item in match.group(1).split(', ')]
        elif match := re.search(r'operation: new = old (.) (.*)', line, flags=re.I):
            monkey.operation = (match.group(1), match.group(2))
        elif match := re.search(r'test: (\w*) .* (\d*)', line, flags=re.I):
            monkey.test = (match.group(1), int(match.group(2)))
        elif match := re.search(r'if true: (\w*) to monkey (\d*)', line, flags=re.I):
            monkey.true_action = (match.group(1), int(match.group(2)))
        elif match := re.search(r'if false: (\w*) to monkey (\d*)', line, flags=re.I):
            monkey.false_action = (match.group(1), int(match.group(2)))
    return monkeys

In [181]:
monkeys = parse(real_data)
for i in range(1, 21):
    # print(f'Round {i:2}')
    monkey_round(monkeys)
    # show_items(monkeys)
ttl = []
print('Monkey business list')
print('----------------------------------------------------')
for i in monkeys:
    print(f'Monkey {i:2}: {monkeys[i].counter:12,}')
    ttl.append(monkeys[i].counter)
ttl.sort(reverse=True)
mb = ttl[0] * ttl[1]
print('----------  ----------- +')
print(f'           {mb:12,}')
print(f'\nOutput for puzzle: {mb}')


Monkey business list
----------------------------------------------------
Monkey  0:          213
Monkey  1:          194
Monkey  2:          245
Monkey  3:          217
Monkey  4:          230
Monkey  5:           46
Monkey  6:          216
Monkey  7:           28
----------  ----------- +
                 56,350

Output for puzzle: 56350


## Part 2

In [250]:
monkeys = supermonkeys()
n = 10000
for i in tqdm(range(n), total=n):
    monkey_round(monkeys)
ttl = []
print('Monkey business list')
print('----------------------------------------------------')
for i in monkeys:
    print(f'Monkey {i:2}: {monkeys[i].counter:12,}')
    ttl.append(monkeys[i].counter)
ttl.sort(reverse=True)
mb = ttl[0] * ttl[1]
print('----------  ----------- +')
print(f'           {mb:12,}')
print(f'\nOutput for puzzle: {mb}')

  0%|          | 0/10000 [00:00<?, ?it/s]

Monkey business list
----------------------------------------------------
Monkey  0:      100,279
Monkey  1:       66,247
Monkey  2:      127,552
Monkey  3:      104,753
Monkey  4:      109,399
Monkey  5:       36,509
Monkey  6:      100,297
Monkey  7:       27,340
----------  ----------- +
           13,954,061,248

Output for puzzle: 13954061248


In [228]:
class SuperMonkey:
    def __init__(self):
        self.counter = 0

    def turn(self):
        '''
        self.calc: lambda calculating new stress level
        self.test: lambda to test value
        self.yes destination if test is true
        self.no destination if test is false
        '''

        throw_list = []
        for item in self.items:
            value = self.calc(item)
            if self.test(value):
                throw_list.append((self.yes, value))
            else:
                throw_list.append((self.no, value))
        self.counter += len(self.items)
        self.items = []
        return throw_list

In [243]:
# test data
def testmonkeys():
    monkeys = {i: SuperMonkey() for i in range(4)}

    monkeys[0].items = [79, 98]
    monkeys[0].calc = lambda x: (x * 19) % 96577
    monkeys[0].test = lambda x: x % 23 == 0
    monkeys[0].yes = 2
    monkeys[0].no = 3

    monkeys[1].items = [54, 65, 75, 74]
    monkeys[1].calc = lambda x: (x + 6) % 96577
    monkeys[1].test = lambda x: x % 19 == 0
    monkeys[1].yes = 2
    monkeys[1].no = 0

    monkeys[2].items = [79, 60, 97]
    monkeys[2].calc = lambda x: (x * x) % 96577
    monkeys[2].test = lambda x: x % 13 == 0
    monkeys[2].yes = 1
    monkeys[2].no = 3

    monkeys[3].items = [74]
    monkeys[3].calc = lambda x: (x + 3) % 96577
    monkeys[3].test = lambda x: x % 17 == 0
    monkeys[3].yes = 0
    monkeys[3].no = 1

    return monkeys

In [248]:
11 * 19 * 5 * 2 * 13 * 7 * 3 * 17

9699690

In [249]:
def supermonkeys():
    monkeys = {i: SuperMonkey() for i in range(8)}

    monkeys[0].items = [97, 81, 57, 57, 91, 61]
    monkeys[0].calc = lambda x: (x * 7) % 9699690
    monkeys[0].test = lambda x: x % 11 == 0
    monkeys[0].yes = 5
    monkeys[0].no = 6

    monkeys[1].items = [88, 62, 68, 90]
    monkeys[1].calc = lambda x: (x * 17) % 9699690
    monkeys[1].test = lambda x: x % 19 == 0
    monkeys[1].yes = 4
    monkeys[1].no = 2

    monkeys[2].items = [74, 87]
    monkeys[2].calc = lambda x: (x + 2) % 9699690
    monkeys[2].test = lambda x: x % 5 == 0
    monkeys[2].yes = 7
    monkeys[2].no = 4

    monkeys[3].items = [53, 81, 60, 87, 90, 99, 75]
    monkeys[3].calc = lambda x: (x + 1) % 9699690
    monkeys[3].test = lambda x: x % 2 == 0
    monkeys[3].yes = 2
    monkeys[3].no = 1

    monkeys[4].items = [57]
    monkeys[4].calc = lambda x: (x + 6) % 9699690
    monkeys[4].test = lambda x: x % 13 == 0
    monkeys[4].yes = 7
    monkeys[4].no = 0

    monkeys[5].items = [54, 84, 91, 55, 59, 72, 75, 70]
    monkeys[5].calc = lambda x: (x * x) % 9699690
    monkeys[5].test = lambda x: x % 7 == 0
    monkeys[5].yes = 6
    monkeys[5].no = 3

    monkeys[6].items = [95, 79, 79, 68, 78]
    monkeys[6].calc = lambda x: (x + 3) % 9699690
    monkeys[6].test = lambda x: x % 3 == 0
    monkeys[6].yes = 1
    monkeys[6].no = 3

    monkeys[7].items = [61, 97, 67]
    monkeys[7].calc = lambda x: (x + 4) % 9699690
    monkeys[7].test = lambda x: x % 17 == 0
    monkeys[7].yes = 0
    monkeys[7].no = 5
    
    return monkeys



In [203]:
m = monkeys[0]

In [205]:
m.turn()

[(3, 1140), (3, 1349), (3, 1539), (3, 1520)]

In [235]:
i

905